In [31]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import String, Column, Integer, DateTime
from sqlalchemy.orm import sessionmaker
from sqlalchemy import func

db_path = 'sqlite:///company1.db'
engine = create_engine(db_path, echo=False)

Base = declarative_base()
Session = sessionmaker(bind=engine)
session = Session()

class Template(Base):
    
    __abstract__ = True
    id = Column(Integer, primary_key=True)
    time_created = Column(DateTime(timezone=True), server_default=func.now())
    time_updated = Column(DateTime(timezone=True), server_default=func.now(), onupdate=func.now())
    
class Employee(Template):
    __tablename__ = "employee"
    
    first_name = Column(String(100))
    last_name = Column(String(100))
    gender = Column(String(32))
    
    def __init__(self, first_name, last_name, gender):
        self.first_name = first_name
        self.last_name = last_name
        self.gender = gender
        
    def __repr__(self):
        return '<Employee %s, %s>' % (self.first_name, self.last_name)
    
Base.metadata.create_all(engine)

akeem = Employee(first_name="Akeem", last_name="Joffer", gender="Male")
session.add(akeem)

session.add_all([
    Employee("Joe", "Smith", "Male"),
    Employee("Jill", "Williams", "Female"),
    Employee("John", "Doe", "Male")
])

session.commit()

In [32]:
myemployees = session.query(Employee).order_by(Employee.id)
for person in myemployees:
    print (person)
    print(person.first_name, person.last_name, person.time_created)

<Employee Akeem, Joffer>
Akeem Joffer 2018-07-13 01:14:09
<Employee Joe, Smith>
Joe Smith 2018-07-13 01:14:09
<Employee Jill, Williams>
Jill Williams 2018-07-13 01:14:09
<Employee John, Doe>
John Doe 2018-07-13 01:14:09
<Employee Akeem, Joffer>
Akeem Joffer 2018-07-13 01:14:17
<Employee Joe, Smith>
Joe Smith 2018-07-13 01:14:18
<Employee Jill, Williams>
Jill Williams 2018-07-13 01:14:18
<Employee John, Doe>
John Doe 2018-07-13 01:14:18


In [26]:
import csv

with open('employees.csv', 'r') as csvfile:
    data = csv.reader(csvfile, delimiter=",")
    for row in data:
        session.add(Employee(first_name=row[0], last_name=row[1], gender=row[2]))
    session.commit()
